# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [26]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp

In [3]:
# create a Spark session
spark = SparkSession \
.builder \
.master('local') \  # change this if it runs in a spark cluster
.appName('sparkify') \
.getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [6]:
data_path = './mini_sparkify_event_data.json'
df = spark.read.json(data_path)

In [7]:
# check the schema of data
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [8]:
# a glimpse of data
df.show(5)

+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|          artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|  Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Boston-Cambridge-...|   PUT|NextSong|1538331630000| 

In [35]:
# convert registration and ts into timestamp
df = df.withColumn('registration_date', to_timestamp(df.registration/1000))
df = df.withColumn('log_date', to_timestamp(df.ts/1000))
df.select('registration_date', 'log_date').show(5)

+-------------------+-------------------+
|  registration_date|           log_date|
+-------------------+-------------------+
|2018-09-28 22:22:42|2018-10-01 00:01:57|
|2018-09-30 18:20:30|2018-10-01 00:03:00|
|2018-09-28 22:22:42|2018-10-01 00:06:34|
|2018-09-30 18:20:30|2018-10-01 00:06:56|
|2018-09-28 22:22:42|2018-10-01 00:11:16|
+-------------------+-------------------+
only showing top 5 rows



In [42]:
num_of_records = df.count()
num_of_no_userid = df.where("userId is null or userId == ''").count()
num_of_no_sessid = df.where("sessionId is null or sessionId == ''").count()
num_of_illegal_gender = df.where("gender <> 'M' and gender <> 'F'").count()
num_of_illegal_length = df.where('length <= 0').count()
max_registration_date = df.agg({'registration_date': 'max'}).collect()[0]['max(registration_date)']
min_registration_date = df.agg({'registration_date': 'min'}).collect()[0]['min(registration_date)']
max_log_date = df.agg({'log_date': 'max'}).collect()[0]['max(log_date)']
min_log_date = df.agg({'log_date': 'min'}).collect()[0]['min(log_date)']

print('number of records: {}'.format(num_of_records))
print('number of records with no user ID: {}'.format(num_of_no_userid))
print('number of records with no session ID: {}'.format(num_of_no_sessid))
print('number of records with illegal gender: {}'.format(num_of_illegal_gender))
print('number of records with length <= 0: {}'.format(num_of_illegal_length))
print('registration date range from {} to {}'.format(min_registration_date, max_registration_date))
print('log date range from {} to {}'.format(min_log_date, max_log_date))

number of records: 286500
number of records with no user ID: 8346
number of records with no session ID: 0
number of records with illegal gender: 0
number of records with length <= 0: 0
registration date range from 2018-03-18 13:44:35 to 2018-11-26 15:49:14
log date range from 2018-10-01 00:01:57 to 2018-12-03 01:11:16


In [44]:
# based on the checking above, records with no user ID will be cleaned
# these records are likely guests, we can simply ignore them
df_cleaned = df.where("userId is not null and userId <> ''")
df_cleaned.count()

278154

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.